The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [ ]:
!pip install -U --pre mxnet-cu101mkl  # updating mxnet to at least v1.6
!pip install d2l  # installing d2l


# Distributed Key-Value Store

:label:`sec_key_value`


KVStore is a place for data sharing. Think of it as a single object shared across different devices (GPUs and computers), where each device can push data in and pull data out.

## Initialization
Let’s consider a simple example: initializing a (int, NDArray) pair into the store, and then pulling the value out:

In [1]:
import d2l
from mxnet import np, npx, kv
npx.set_np()

In [2]:
np.ones((2,3))

array([[1., 1., 1.],
       [1., 1., 1.]])

In [3]:
help(kv)

Help on module mxnet.kvstore in mxnet:

NAME
    mxnet.kvstore - Key value store interface of MXNet for parameter synchronization.

CLASSES
    builtins.object
        KVStore
    
    class KVStore(builtins.object)
     |  KVStore(handle)
     |  
     |  A key-value store for synchronization of values, over multiple devices.
     |  
     |  Methods defined here:
     |  
     |  __del__(self)
     |  
     |  __init__(self, handle)
     |      Initializes a new KVStore.
     |      
     |      Parameters
     |      ----------
     |      handle : KVStoreHandle
     |          `KVStore` handle of C API.
     |  
     |  init(self, key, value)
     |      Initializes a single or a sequence of key-value pairs into the store.
     |      
     |      For each key, one must `init` it before calling `push` or `pull`.
     |      When multiple workers invoke `init` for the same key, only
     |      the value supplied by worker with rank `0` is used. This function returns
     |      aft

In [4]:
kv = kv.create('local') # create a local kv store.
shape = (2,3)
kv.init(3, np.ones(shape)*2)
a = np.zeros(shape)
kv.pull(3, out = a)
print(a)

[[2. 2. 2.]
 [2. 2. 2.]]


## Push, Aggregate, and Update

For any key that has been initialized, you can push a new value with the same shape to the key:

In [5]:
kv.push(3, np.ones(shape)*8)
kv.pull(3, out = a) # pull out the value
print(a.asnumpy())

[[8. 8. 8.]
 [8. 8. 8.]]


The data for pushing can be stored on any device. Furthermore, you can push multiple values into the same key, where KVStore will first sum all of these values and then push the aggregated value. Here we will just demonstrate pushing a list of values on CPU. Please note summation only happens if the value list is longer than one

In [6]:
contexts = [npx.cpu(i) for i in range(4)]
b = [np.ones(shape, ctx=ctx) for ctx in contexts]
kv.push(3, b)
kv.pull(3, out = a)
print(a)

[[4. 4. 4.]
 [4. 4. 4.]]


For each push, KVStore combines the pushed value with the value stored using an updater. The default updater is ASSIGN. You can replace the default to control how data is merged:

In [7]:
def update(key, input, stored):
    print("update on key: %d" % key)
    stored += input * 2
kv._set_updater(update)
kv.pull(3, out=a)
print(a)

[[4. 4. 4.]
 [4. 4. 4.]]


In [8]:
kv.push(3, np.ones(shape))
kv.pull(3, out=a)
print(a)

update on key: 3
[[6. 6. 6.]
 [6. 6. 6.]]


## Pull

You’ve already seen how to pull a single key-value pair. Similarly, to push, you can pull the value onto several devices with a single call:

In [9]:
b = [np.ones(shape, ctx=ctx) for ctx in contexts]
kv.pull(3, out = b)
print(b[1])

[[6. 6. 6.]
 [6. 6. 6.]]


## Handle a List of Key-Value Pairs

All operations introduced so far involve a single key. KVStore also provides an interface for a list of key-value pairs.

For a single device:

In [10]:
keys = [5, 7, 9]
kv.init(keys, [np.ones(shape)]*len(keys))
kv.push(keys, [np.ones(shape)]*len(keys))
b = [np.zeros(shape)]*len(keys)
kv.pull(keys, out = b)
print(b[1])

update on key: 5
update on key: 7
update on key: 9
[[3. 3. 3.]
 [3. 3. 3.]]


For multiple devices:

In [11]:
b = [[np.ones(shape, ctx=ctx) for ctx in contexts]] * len(keys)
kv.push(keys, b)
kv.pull(keys, out = b)
print(b[1][1])

update on key: 5
update on key: 7
update on key: 9
[[11. 11. 11.]
 [11. 11. 11.]]
